In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv('data/events_up_to_01062018.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
df.head(5)

In [ ]:
test = df.groupby('person')['timestamp'].value_counts().to_frame()

In [ ]:
test.columns = ['contar']

In [ ]:
test.reset_index(inplace=True)

In [ ]:
test

In [ ]:
test.sort_values(['person', 'timestamp'], ascending=True,inplace=True)

In [ ]:
test

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display

persona_antes=None
test["conte_10"] = False

f = IntProgress(min=0, max=2023613) # instantiate the bar
display(f) # display the bar


usuario_ya_con_conte = False   
for i, fila in test.iterrows():
    f.value += 1 # signal to increment the progress bar
    contar=0
    if persona_antes is None or persona_antes != fila["person"]:
        usuario_ya_con_conte=False
        persona_antes=fila["person"]
        df_aux = test[test["person"]==fila["person"]]
    if usuario_ya_con_conte: continue
        
    for i2, fila2 in df_aux.iterrows():
        if (abs(fila['timestamp'] - fila2['timestamp'])).seconds < 60*20: # 20 minutos
            contar+=test.at[i,"contar"]
        else:
            break
            
        if contar==10:
            test.at[i,"conte_10"] = True
            usuario_ya_con_conte = True
            break
    

In [ ]:
test.to_csv("data/salvo.csv", sep=',')

In [ ]:
test[test["conte_10"]==True].to_csv("data/conte_10.csv",sep=",")

In [ ]:
test

In [ ]:
usuarios_solo_con_true = test[test["conte_10"]==True]

In [ ]:
personas_sin_repetir = df.drop_duplicates(subset="person", keep='first')

In [ ]:
personas_sin_repetir = personas_sin_repetir.loc[:,"person"]

In [ ]:
personas_sin_repetir = personas_sin_repetir.to_frame()

In [ ]:
type(personas_sin_repetir.at[0,"person"])

In [ ]:
usuarios_solo_con_true

In [ ]:
personas_sin_repetir.set_index("person",inplace=True)
usuarios_solo_con_true.set_index("person",inplace=True)

In [ ]:
df_conte = personas_sin_repetir.join(usuarios_solo_con_true,on="person",how="left")

In [ ]:
df_final = df_conte.loc[:,"conte_10"].to_frame().fillna(False)

In [ ]:
df_final.to_csv("data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv", sep=',')

In [ ]:
df_final